In [1]:
# in order to make the source code easy to read and modify I've placed those modules in a separate files    
import DataExtraction   # this module is responsible for extractin the email's body from the mailbox [without signature]
import DataAnalysis     # this module is responsible for parsing the email's body and extracting the required data

def main():
    # Specify which mailbox/store to read, exactly matching 'store.DisplayName'
        # this part of the source code can be modified to read from a different mailbox or subfolder
        # in case we have to run it on different user's machine or mailbox.
    mailbox_name    = "n.yumer@my.bpp.com"  # for anonimity reasons I've removed the mailbox name
    subfolder_name  = "ROMIC"

    # Read emails from the subfolder
    all_bodies = DataExtraction.read_outlook_subfolder_stores(mailbox_name, subfolder_name)

    # For demonstration purposes of this project I print the result, but in the real scenario
    # I will wreite the resuts in dedicated excell file for the interested parites to review and use.
    
    parsed_results = DataAnalysis.parse_emails(all_bodies)
    print("RESULTS:")
    print(f"  Monitors: {parsed_results['monitors']}")

    # [NEW] Print the recognized desktop models
    print("  Desktops:")
    for model, qty in parsed_results['desktops'].items():
        print(f"    Model {model}: {qty}")

    # [NEW] Print the recognized laptop models
    print("  Laptops:")
    for model, qty in parsed_results['laptops'].items():
        print(f"    Model {model}: {qty}")

    # [NEW] Print the recognized phone models
    print("  Phones:")
    for model, qty in parsed_results['phones'].items():
        print(f"    Model {model}: {qty}")

    print("  Bags:")
    for bag_type, qty in parsed_results['bags'].items():
        print(f"    {bag_type} bag: {qty}")

    print("  Chargers:")
    for charger_label, qty in parsed_results['chargers'].items():
        print(f"    {charger_label}: {qty}")

    print(f"  Docks: {parsed_results['docks']}")

    print("  Headsets:")
    for headset_label, qty in parsed_results['headsets'].items():
        print(f"    {headset_label}: {qty}")

    # [NEW] Print the unparsable lines with sender and time
    print("  Unparsable Lines:")
    for unparsable in parsed_results['unparsable_lines']:
        line = unparsable['line']
        sender = unparsable['sender']
        time = unparsable['received_time']
        print(f"    Line: \"{line}\", Sender: {sender}, Received Time: {time}")
if __name__ == "__main__":
    main()



AttributeError: 'tuple' object has no attribute 'upper'